In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-24"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12018,2024-04-24,Itália Série A1 Feminina,15:30,Campobasso F,San Giovanni F,1.61,2.22,123.5,1.85,1.85,0.0,0.00,0.00,pdUyftO9,0.621118,0.450450,0.540541,0.540541,0.071568,125.780,86.100706,0.684534,2.4,1.341641,0.559017,277.30,1.900,1.568502,0.825528,38.0,122.276,71.309220,0.583182,1.8,1.643168,0.912871,81.28,2.068,1.371703,0.663299,2.0,59,64,4.70,1.27,132.292,126.192,0.225240,0.000000,NaN,0.87,0.174,3.505747,0.000000,0.0,0.000000,1.04,0.208,5.865385,0.000000,0.0,0.000000
12019,2024-04-24,Nova Zelândia Nbl,23:00,Otago Nuggets,Canterbury Rams,1.61,2.22,185.5,1.87,1.79,-3.5,1.94,2.49,xMulHHBG,0.621118,0.450450,0.534759,0.558659,0.071568,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,71,104,2.68,2.19,0.000,0.000,0.225240,0.030912,0.17558,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12020,2024-04-24,Nova Zelândia Nbl,04:30,Auckland Tuatara,Franklin Bulls,1.64,2.17,176.5,1.87,1.79,0.0,0.00,0.00,vyNpIcdA,0.609756,0.460829,0.534759,0.558659,0.070586,140.504,39.521755,0.281286,2.4,1.341641,0.559017,94.94,1.502,0.317443,0.211347,75.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,94,105,1.01,1.43,0.000,0.000,0.196728,0.030912,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12021,2024-04-24,México Cibacopa,00:15,Venados de Mazatlan,Astros,2.76,1.41,177.5,1.83,1.87,0.0,0.00,0.00,b3SedT4O,0.362319,0.709220,0.546448,0.534759,0.071539,221.216,61.283354,0.277029,2.4,1.341641,0.559017,202.10,2.442,0.745902,0.305447,-4.0,132.688,15.043474,0.113375,1.8,1.643168,0.912871,126.14,1.350,0.198872,0.147312,42.0,94,106,2.15,1.19,145.944,148.764,0.457839,0.015289,NaN,0.94,0.188,9.361702,0.586343,0.7,0.113657,-0.15,-0.030,-13.666667,0.666111,0.6,-0.066111
12022,2024-04-24,México Cibacopa,00:15,Rayos de Hermosillo,Pioneros de Los Mochis,1.35,3.02,179.5,1.84,1.86,0.0,0.00,0.00,C6WaemKU,0.740741,0.331126,0.543478,0.537634,0.071867,197.968,35.495917,0.179301,1.2,1.643168,1.369306,162.84,2.134,0.413316,0.193681,-19.0,245.616,106.621091,0.434097,1.2,1.643168,1.369306,406.08,2.798,1.264425,0.451903,-31.0,92,94,1.77,4.32,144.260,244.660,0.540443,0.007644,NaN,-2.91,-0.582,-0.601375,0.672191,0.5,-0.172191,-0.98,-0.196,-10.306122,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12097,2024-04-24,Espanha Liga Feminina,14:30,Estudiantes F,Zaragoza F,2.24,1.59,130.5,1.82,1.88,0.0,0.00,0.00,AoQsRdcD,0.446429,0.628931,0.549451,0.531915,0.075359,150.338,64.062501,0.426123,0.6,1.341641,2.236068,135.45,2.288,1.053575,0.460479,-15.0,84.752,10.851863,0.128043,3.0,0.000000,0.000000,84.00,1.146,0.092628,0.080827,59.0,63,80,2.15,1.05,153.884,109.428,0.240010,0.022933,NaN,-1.37,-0.274,-4.525547,0.674622,0.7,0.025378,1.91,0.382,1.544503,0.644271,0.8,0.155729
12098,2024-04-24,Eslováquia Extraliga Feminina,13:00,Slavia Banska Bystrica F,Piestanske Cajky F,2.34,1.55,139.5,1.86,1.84,0.0,0.00,0.00,xzJtZnAm,0.427350,0.645161,0.537634,0.543478,0.072512,96.862,46

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
12030,13:30,Israel Liga Leumit,Gilboa Galil,Maccabi Rishon,1.66,Back Home
12074,14:00,Europa Euroliga,Monaco,Fenerbahce,1.67,Back Home
